# Projet 6
## Description du projet

## Configuration

In [55]:
# New packages


# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import sys, os
import skimage.io as io
import pandas as pd
import numpy as np
import nltk
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import random
from nltk.tokenize import RegexpTokenizer
import string
from nltk.corpus import stopwords ; nltk.download('stopwords') ; nltk.download('wordnet')
from nltk.stem import PorterStemmer, WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\C0002630\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\C0002630\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Variables globales
path_to_csv = os.path.join("data", "flipkart_com-ecommerce_sample_1050.csv")
path_to_images_folder = os.path.join("data", "images")

## Partie 1 : Chargement des données

In [3]:
df = pd.read_csv(path_to_csv)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  1050 non-null   object 
 1   crawl_timestamp          1050 non-null   object 
 2   product_url              1050 non-null   object 
 3   product_name             1050 non-null   object 
 4   product_category_tree    1050 non-null   object 
 5   pid                      1050 non-null   object 
 6   retail_price             1049 non-null   float64
 7   discounted_price         1049 non-null   float64
 8   image                    1050 non-null   object 
 9   is_FK_Advantage_product  1050 non-null   bool   
 10  description              1050 non-null   object 
 11  product_rating           1050 non-null   object 
 12  overall_rating           1050 non-null   object 
 13  brand                    712 non-null    object 
 14  product_specifications  

In [4]:
# for idx in range(10):
#     img_path = os.path.join("data", "images", df.loc[idx, "image"])
#     img = io.imread(img_path)
#     print(df.loc[idx, "product_name"])
#     print(df.loc[idx, "product_category_tree"])
#     # print(df.loc[idx, "description"])
#     plt.imshow(img)
#     plt.show()

# A. Categories

In [103]:
def count_categoriy(category_tree):
    # category_tree = category_tree.replace('["', '')
    # category_tree = category_tree.replace('"]', '')
    category_tree = category_tree.replace('>>', '>')
    counter = 1
    for elt in category_tree:
        if elt == ">":
            counter += 1
    return counter

df["nb_of_categories"] = df["product_category_tree"].apply(count_categoriy)
print(df["nb_of_categories"].min())
print(df["nb_of_categories"].max())

2
7


# B. NLP

## B.1 Preprocessing

In [83]:
# Tokenized corpus
tokenizer = nltk.RegexpTokenizer(r"\w+")
tokenized_corpus = [tokenizer.tokenize(doc) for doc in df["description"]]

# Remove punctuation
for text in tokenized_corpus:
    for token in text:
        if token in string.punctuation:
            text.remove(token)

# Stemming
# Initialize the stemmer
stemmer = PorterStemmer()
stemmer = WordNetLemmatizer()
# Stem the tokens
for i in range(len(tokenized_corpus)):
    for j in range(len(tokenized_corpus[i])):
        tokenized_corpus[i][j] = stemmer.lemmatize(tokenized_corpus[i][j]).lower()

# Count each words
bow = [word for text in tokenized_corpus for word in text]
bow = Counter(bow)
bow = dict(sorted(bow.items(), key=lambda item: item[1], reverse=True))
print("20 most occurences:")
for i in range(20):
    word = list(bow.keys())[i]
    print(f"{word:<9}: {bow[word]}")
print("==> What is rs? Indian Rupees (money)\n")

# Drop stopwords
stop_words = set(stopwords.words('english'))
for text in tokenized_corpus:
    for word in text:
        if word in stop_words:
            text.remove(word)

# Count each words
bow = [word for text in tokenized_corpus for word in text]
bow = Counter(bow)
bow = dict(sorted(bow.items(), key=lambda item: item[1], reverse=True))
print("20 most occurences after removing stopwords:")
for i in range(20):
    word = list(bow.keys())[i]
    print(f"{word:<9}: {bow[word]}")

20 most occurences:
of       : 1751
for      : 1436
the      : 1369
and      : 1332
to       : 1058
in       : 1047
rs       : 911
only     : 890
with     : 843
on       : 835
a        : 814
at       : 714
your     : 656
is       : 638
free     : 618
buy      : 581
products : 577
cm       : 567
delivery : 567
genuine  : 564
==> What is rs? Indian Rupees (money)

20 most occurences after removing stopwords:
rs       : 911
the      : 772
free     : 618
buy      : 581
products : 577
cm       : 567
delivery : 567
genuine  : 564
shipping : 564
cash     : 564
replacement: 559
1        : 554
day      : 549
price    : 541
30       : 498
flipkart : 481
com      : 473
guarantee: 471
mug      : 406
online   : 396
